## Bibliothèques

In [3]:
import sys
import os
import numpy as numpy
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

## Loading Files

### Set up Path

In [8]:
DIR_PATH = os.getcwd()
DATA_PATH = os.path.join(DIR_PATH + os.sep, "data")
WRLD_PATH = os.path.join(DATA_PATH + os.sep, "World")
BRL_PATH =  os.path.join(WRLD_PATH + os.sep, "Berlin")

/Users/pierreromon/Git/AirBnBAnalysis/AirbnbAnalysis/data/World/Berlin


### Load *.csv

In [12]:
listing = pd.read_csv(BRL_PATH + "/listings.csv", low_memory=False)
review = pd.read_csv(BRL_PATH + "/reviews.csv", low_memory=False)
#calendar = pd.read_csv(BRL_PATH + "/calendar.csv", low_memory=False)

## Clean files

In [15]:
listing.head(5)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2015,Berlin-Mitte Value! Quiet courtyard/very central,2217,Ian,Mitte,Brunnenstr. Süd,52.534537,13.402557,Entire home/apt,60,4,118,2018-10-28,3.76,4,141
1,2695,Prenzlauer Berg close to Mauerpark,2986,Michael,Pankow,Prenzlauer Berg Nordwest,52.548513,13.404553,Private room,17,2,6,2018-10-01,1.42,1,0
2,3176,Fabulous Flat in great Location,3718,Britta,Pankow,Prenzlauer Berg Südwest,52.534996,13.417579,Entire home/apt,90,62,143,2017-03-20,1.25,1,220
3,3309,BerlinSpot Schöneberg near KaDeWe,4108,Jana,Tempelhof - Schöneberg,Schöneberg-Nord,52.498855,13.349065,Private room,26,5,25,2018-08-16,0.39,1,297
4,7071,BrightRoom with sunny greenview!,17391,Bright,Pankow,Helmholtzplatz,52.543157,13.415091,Private room,42,2,197,2018-11-04,1.75,1,26


In [20]:
listing = listing.loc[:,['id', 'room_type', 'host_id', 'host_since', 'host_response_rate', 'reviews_per_month', 'review_scores_value', 'review_scores_location', 'review_scores_communication', 
'review_scores_checkin', 'review_scores_cleanliness', 'review_scores_rating', 'price', 'cleaning_fee', 'zipcode', 'number_of_reviews']]
listing = listing.dropna(subset=['id', 'room_type', 'host_id', 'host_since', 'reviews_per_month', 'price', 'zipcode' ])
listing.head(5)

,id,room_type,host_id,host_since,host_response_rate,reviews_per_month,review_scores_value,review_scores_location,review_scores_communication,review_scores_checkin,review_scores_cleanliness,review_scores_rating,price,cleaning_fee,zipcode,number_of_reviews


## Analysis

In [ ]:
### 